# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera
### by Hydilla

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to compare the neighborhoods of the cities and determine how similar or dissimilar they are.We will look into the cities we have analyzed before and add more cities to the analysis.

All cities in the project are the economical or cultural center of the country,we will find how similiar they are and how the segementions will distrubute.

## Data <a name="data"></a>

Based on definition of our problem, we will look into three cities:
* New York City
* Toronto
* Los Angeles

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Berlin center will be obtained using **Google Maps API geocoding** of well known Berlin location (Alexanderplatz)

### Neighborhood Candidates

Since different cities have different ways to obtain data about the location and other geo-data,so the preprocessing step will be put after basic processing.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

#### Data
##### NY city

In [27]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [28]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [29]:
ny_neighborhoods_data = newyork_data['features']

In [30]:
ny_neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [31]:
# define the dataframe columns
ny_column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods = pd.DataFrame(columns=ny_column_names)

In [32]:
ny_neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [33]:
for data in ny_neighborhoods_data:
    borough = ny_neighborhood_name = data['properties']['borough'] 
    ny_neighborhood_name = data['properties']['name']
        
    ny_neighborhood_latlon = data['geometry']['coordinates']
    ny_neighborhood_lat = ny_neighborhood_latlon[1]
    ny_neighborhood_lon = ny_neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': ny_neighborhood_name,
                                          'Latitude': ny_neighborhood_lat,
                                          'Longitude': ny_neighborhood_lon}, ignore_index=True)

In [34]:
ny_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(ny_neighborhoods['Borough'].unique()),
        ny_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


##### Toronto

In [36]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_133dc73784bb485cae2f072256e41b55 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Fv9JE6L4y7dEVKBNw-tKa-GXJmZBo7vsvy9RyKHDIAl5',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_133dc73784bb485cae2f072256e41b55.get_object(Bucket='courseracapstone-donotdelete-pr-mrguwl6yxgocoj',Key='trodata.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [37]:
df=df_data_1
y=df[df['Borough'].str.contains('Not assigned')]
test=list(y['Borough'])
test1=list(df['Borough'])
ret=list(set(test1)^set(test))
result=df[df['Borough'].isin(ret)]
result.sort_values(by="Neighbourhood" , ascending=True).head()

,Postalcode,Borough,Neighbourhood
67,M5H,Downtown Toronto,Adelaide
179,M1S,Scarborough,Agincourt
204,M1V,Scarborough,Agincourt North
227,M9V,Etobicoke,Albion Gardens
242,M8W,Etobicoke,Alderwood


In [33]:
result.shape

(210, 3)

In [38]:
result.loc[9,'Neighbourhood']=result.loc[9,'Borough']
result.loc[9,'Neighbourhood']

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


"Queen's Park"

In [39]:
def ab(df):
    return ','.join(df.values)
result=result.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(ab)
result=result.reset_index()
result.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [40]:
body = client_133dc73784bb485cae2f072256e41b55.get_object(Bucket='courseracapstone-donotdelete-pr-mrguwl6yxgocoj',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
df_data_2 = df_data_2.rename(columns={'Postal Code':'Postalcode'})
df_data_2.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
outcome=pd.merge(result,df_data_2,on='Postalcode')
outcome.sort_values(by="Neighbourhood" , ascending=True).head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
58,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
12,M1S,Scarborough,Agincourt,43.794200,-79.262029
14,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484


##### Los Angeles

In [4]:

body = client_133dc73784bb485cae2f072256e41b55.get_object(Bucket='courseracapstone-donotdelete-pr-mrguwl6yxgocoj',Key='ladata.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Neighborhood,LALO
0,Agoura Hills,"34.13600,-118.77500"
1,Alhambra,"34.09500,-118.12700"
2,Arcadia,"33.98095,-118.14697"
3,Artesia,"33.86607,-118.08294"
4,Avalon,"33.34300,-118.32800"


In [5]:
lalo=df_data_0['LALO'].str.split(',',expand=True)

In [6]:
lalo.columns=['Latitude','Longitude']

In [7]:
lalo.head()

,Latitude,Longitude
0,34.13600,-118.77500
1,34.09500,-118.12700
2,33.98095,-118.14697
3,33.86607,-118.08294
4,33.34300,-118.32800


In [17]:
lalo.dtypes

Latitude     object
Longitude    object
dtype: object

In [23]:
lalo['Latitude']=lalo['Latitude'].astype("float")
lalo['Longitude']=lalo['Longitude'].astype("float")

In [24]:
la_data=df_data_0.join(lalo)
la_data=la_data.drop(['LALO'],axis=1)

In [25]:
la_data.head()

,Neighborhood,Latitude,Longitude
0,Agoura Hills,34.13600,-118.77500
1,Alhambra,34.09500,-118.12700
2,Arcadia,33.98095,-118.14697
3,Artesia,33.86607,-118.08294
4,Avalon,33.34300,-118.32800


## Methodology <a name="methodology"></a>

In this project we will analyze the segematations of the three cities and find out the similiarities of the three parts.We will use the Foursquare information and machine learning methodology to divide the boroughs in these three cities.

## Analysis <a name="analysis"></a>

Let's show the maps and dots on them.

First we visualize the NY 's map

In [43]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [45]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ny_neighborhoods['Latitude'], ny_neighborhoods['Longitude'], ny_neighborhoods['Borough'], ny_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Second we show the Toronto condition

In [46]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[43.40,-79.25], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(outcome['Latitude'], outcome['Longitude'], outcome['Borough'], outcome['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Third we show the Los angeles's condition

In [26]:
# create map of Los Angeles using latitude and longitude values
map_loso = folium.Map(location=[34.03,-118.15], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(la_data['Latitude'], la_data['Longitude'],  la_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_loso)  
    
map_loso

OK, we have show the three districts we want to compare,they are all well-developed cities and we will segematate them one by one.

#### Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'KWGFHWIGGPCXJ11XJJWSGW50NCWA0D1HML4AIAV0LE1LM1OF' # your Foursquare ID
CLIENT_SECRET = 'QQAMJOX5VZRIK340VQ4ITUTT3PVWHPFUUWRRWFQGKAYM2XXV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KWGFHWIGGPCXJ11XJJWSGW50NCWA0D1HML4AIAV0LE1LM1OF
CLIENT_SECRET:QQAMJOX5VZRIK340VQ4ITUTT3PVWHPFUUWRRWFQGKAYM2XXV


In [48]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Analyze the districts

NY city

In [49]:
ny_venues = getNearbyVenues(names=ny_neighborhoods['Neighborhood'],
                                   latitudes=ny_neighborhoods['Latitude'],
                                   longitudes=ny_neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [51]:
print(ny_venues.shape)
ny_venues.head()

(7858, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [54]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,28,28,28,28,28,28
Annadale,10,10,10,10,10,10
Arden Heights,4,4,4,4,4,4
Arlington,7,7,7,7,7,7
Arrochar,20,20,20,20,20,20
Arverne,18,18,18,18,18,18
Astoria,50,50,50,50,50,50
Astoria Heights,12,12,12,12,12,12
Auburndale,18,18,18,18,18,18


In [59]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 409 uniques categories.


In [55]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot= ny_onehot[fixed_columns]

ny_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pa

In [61]:
ny_onehot.shape

(7869, 409)

In [56]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pa

In [63]:
ny_grouped.shape

(300, 409)

In [57]:
num_top_venues = 10

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                 venue  freq
0          Pizza Place  0.14
1          Supermarket  0.11
2        Deli / Bodega  0.07
3   Chinese Restaurant  0.07
4         Dessert Shop  0.04
5    Martial Arts Dojo  0.04
6          Gas Station  0.04
7  Fried Chicken Joint  0.04
8   Spanish Restaurant  0.04
9       Breakfast Spot  0.04


----Annadale----
              venue  freq
0       Pizza Place   0.3
1              Park   0.1
2             Diner   0.1
3    Cosmetics Shop   0.1
4        Restaurant   0.1
5            Bakery   0.1
6        Sports Bar   0.1
7     Train Station   0.1
8       Yoga Studio   0.0
9  Pedestrian Plaza   0.0


----Arden Heights----
                           venue  freq
0                       Pharmacy  0.25
1                    Coffee Shop  0.25
2                           Pool  0.25
3                    Pizza Place  0.25
4                    Yoga Studio  0.00
5           Outdoor Supply Store  0.00
6               Pedestrian Plaza  0.00
7                      

                   venue  freq
0        Harbor / Marina  0.29
1         Baseball Field  0.14
2             Donut Shop  0.14
3           Hockey Field  0.14
4     Athletics & Sports  0.14
5             Playground  0.14
6           Optical Shop  0.00
7        Organic Grocery  0.00
8  Performing Arts Venue  0.00
9       Pedestrian Plaza  0.00


----Blissville----
                 venue  freq
0        Deli / Bodega  0.12
1                Hotel  0.12
2           Donut Shop  0.12
3           Restaurant  0.06
4       Clothing Store  0.06
5         Skating Rink  0.06
6  Sporting Goods Shop  0.06
7            Cafeteria  0.06
8          Bus Station  0.06
9        Movie Theater  0.06


----Bloomfield----
                           venue  freq
0              Recreation Center   0.2
1                           Park   0.2
2                     Theme Park   0.2
3                       Bus Stop   0.2
4                 Discount Store   0.2
5           Outdoor Supply Store   0.0
6          Performing Art

----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1                  Bar  0.05
2    French Restaurant  0.05
3  American Restaurant  0.05
4   Chinese Restaurant  0.05
5   Seafood Restaurant  0.05
6          Pizza Place  0.02
7       Cosmetics Shop  0.02
8        Metro Station  0.02
9         Cycle Studio  0.02


----Charleston----
            venue  freq
0   Big Box Store  0.07
1  Cosmetics Shop  0.07
2       Pet Store  0.03
3  Hardware Store  0.03
4     Pizza Place  0.03
5  Scenic Lookout  0.03
6      Shoe Store  0.03
7     Coffee Shop  0.03
8   Movie Theater  0.03
9  Clothing Store  0.03


----Chelsea----
                 venue  freq
0                Hotel  0.05
1              Theater  0.05
2            Nightclub  0.04
3       Ice Cream Shop  0.04
4               Bakery  0.04
5   Italian Restaurant  0.04
6          Coffee Shop  0.04
7  Japanese Restaurant  0.04
8   Seafood Restaurant  0.04
9       Sandwich Place  0.04


----Chinatown----
                

                  venue  freq
0  Caribbean Restaurant  0.09
1           Pizza Place  0.07
2          Liquor Store  0.04
3         Deli / Bodega  0.04
4            Donut Shop  0.04
5     Food & Drink Shop  0.02
6        Hardware Store  0.02
7  Gym / Fitness Center  0.02
8         Grocery Store  0.02
9   Fried Chicken Joint  0.02


----Dongan Hills----
                  venue  freq
0           Pizza Place  0.09
1         Deli / Bodega  0.09
2    Italian Restaurant  0.09
3              Pharmacy  0.09
4     Convenience Store  0.04
5      Basketball Court  0.04
6                  Bank  0.04
7  Fast Food Restaurant  0.04
8                   Bar  0.04
9        Sandwich Place  0.04


----Douglaston----
                  venue  freq
0         Deli / Bodega  0.10
1                  Bank  0.10
2           Yoga Studio  0.05
3        Ice Cream Shop  0.05
4        Shipping Store  0.05
5   Shanghai Restaurant  0.05
6  Fast Food Restaurant  0.05
7              Pharmacy  0.05
8    Chinese Restaurant  0

                           venue  freq
0                          Plaza  0.33
1                          River  0.33
2                    Bus Station  0.33
3              Other Repair Shop  0.00
4               Pedestrian Plaza  0.00
5                           Park  0.00
6  Paper / Office Supplies Store  0.00
7           Pakistani Restaurant  0.00
8              Paella Restaurant  0.00
9                   Outlet Store  0.00


----Financial District----
                  venue  freq
0           Coffee Shop  0.08
1           Pizza Place  0.08
2            Steakhouse  0.06
3   American Restaurant  0.06
4            Food Truck  0.04
5  Gym / Fitness Center  0.04
6         Jewelry Store  0.04
7                   Gym  0.04
8          Gourmet Shop  0.02
9   Fried Chicken Joint  0.02


----Flatbush----
                       venue  freq
0       Caribbean Restaurant  0.10
1         Mexican Restaurant  0.10
2                Coffee Shop  0.10
3                       Bank  0.10
4                 

...

                venue  freq
0   Korean Restaurant  0.12
1  Chinese Restaurant  0.12
2         Yoga Studio  0.08
3          Donut Shop  0.08
4          Bagel Shop  0.04
5         Pizza Place  0.04
6      Sandwich Place  0.04
7            Pharmacy  0.04
8         Coffee Shop  0.04
9  Persian Restaurant  0.04


----Oakwood----
                           venue  freq
0                         Lawyer  0.33
1                            Bar  0.33
2                     Playground  0.33
3                    Yoga Studio  0.00
4          Performing Arts Venue  0.00
5               Pedestrian Plaza  0.00
6                           Park  0.00
7  Paper / Office Supplies Store  0.00
8           Pakistani Restaurant  0.00
9              Paella Restaurant  0.00


----Ocean Hill----
                             venue  freq
0                    Deli / Bodega  0.16
1                    Grocery Store  0.08
2                       Playground  0.08
3  Southern / Soul Food Restaurant  0.08
4                  

                  venue  freq
0                 Hotel  0.20
1      Basketball Court  0.07
2             Roof Deck  0.07
3           Beer Garden  0.07
4                  Park  0.07
5  Gym / Fitness Center  0.07
6        Scenic Lookout  0.07
7        Sandwich Place  0.07
8    Athletics & Sports  0.07
9              Platform  0.07


----Randall Manor----
                           venue  freq
0                       Bus Stop  0.50
1                     Playground  0.25
2                           Park  0.25
3                    Yoga Studio  0.00
4          Performing Arts Venue  0.00
5               Pedestrian Plaza  0.00
6  Paper / Office Supplies Store  0.00
7           Pakistani Restaurant  0.00
8              Paella Restaurant  0.00
9                   Outlet Store  0.00


----Ravenswood----
                 venue  freq
0   Chinese Restaurant  0.12
1        Grocery Store  0.08
2                Diner  0.04
3    Indian Restaurant  0.04
4   Spanish Restaurant  0.04
5  Fried Chicken Joint

                           venue  freq
0                   Burger Joint   0.2
1                            Gym   0.2
2                    Coffee Shop   0.2
3            American Restaurant   0.2
4                    Golf Course   0.2
5          Performing Arts Venue   0.0
6               Pedestrian Plaza   0.0
7                           Park   0.0
8  Paper / Office Supplies Store   0.0
9           Pakistani Restaurant   0.0


----Soho----
                 venue  freq
0       Clothing Store  0.12
1        Women's Store  0.08
2  Sporting Goods Shop  0.06
3           Shoe Store  0.06
4          Men's Store  0.06
5             Boutique  0.06
6               Bakery  0.04
7   Salon / Barbershop  0.04
8    Accessories Store  0.04
9            Gift Shop  0.04


----Somerville----
                           venue  freq
0                           Park   1.0
1                    Yoga Studio   0.0
2           Outdoor Supply Store   0.0
3             Persian Restaurant   0.0
4          Performing

                    venue  freq
0           Deli / Bodega  0.14
1      Italian Restaurant  0.14
2  Thrift / Vintage Store  0.14
3      Mexican Restaurant  0.14
4          Cosmetics Shop  0.14
5                Bus Stop  0.14
6      Miscellaneous Shop  0.14
7             Yoga Studio  0.00
8             Outlet Mall  0.00
9   Performing Arts Venue  0.00


----Travis----
                venue  freq
0               Hotel  0.18
1       Bowling Alley  0.12
2       Deli / Bodega  0.12
3                Park  0.06
4    Business Service  0.06
5        Cycle Studio  0.06
6         Sports Club  0.06
7      Baseball Field  0.06
8         Pizza Place  0.06
9  Spanish Restaurant  0.06


----Tribeca----
                 venue  freq
0                 Café  0.06
1             Wine Bar  0.06
2  American Restaurant  0.06
3     Greek Restaurant  0.06
4                  Spa  0.06
5           Steakhouse  0.04
6                Hotel  0.04
7          Men's Store  0.04
8                 Park  0.04
9          Yoga

                venue  freq
0            Pharmacy  0.10
1   Martial Arts Dojo  0.05
2         Pizza Place  0.05
3              Bakery  0.05
4          Bagel Shop  0.05
5       Grocery Store  0.05
6        Home Service  0.05
7  Chinese Restaurant  0.05
8    Sushi Restaurant  0.05
9        Liquor Store  0.05


----Woodside----
                       venue  freq
0              Grocery Store  0.10
1                     Bakery  0.08
2  Latin American Restaurant  0.08
3            Thai Restaurant  0.06
4                        Pub  0.06
5        American Restaurant  0.06
6             Ice Cream Shop  0.04
7               Gourmet Shop  0.04
8        Filipino Restaurant  0.04
9                        Bar  0.04


----Yorkville----
                  venue  freq
0    Italian Restaurant  0.12
1                   Gym  0.08
2           Coffee Shop  0.06
3         Deli / Bodega  0.06
4             Wine Shop  0.04
5                  Park  0.04
6        Sandwich Place  0.02
7                  Café  0.0

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
ny_neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Chinese Restaurant,Deli / Bodega,Check Cashing Service,Gas Station,Breakfast Spot,Bus Station,Food,Spanish Restaurant
1,Annadale,Pizza Place,Park,Train Station,Cosmetics Shop,Sports Bar,Bakery,Diner,Restaurant,Exhibit,Factory
2,Arden Heights,Pizza Place,Pool,Pharmacy,Coffee Shop,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory
3,Arlington,Bus Stop,Food,Intersection,Deli / Bodega,American Restaurant,Boat or Ferry,Fish & Chips Shop,Event Space,Exhibit,Factory
4,Arrochar,Deli / Bodega,Italian Restaurant,Bus Stop,Pizza Place,Food Truck,Sandwich Place,Mediterranean Restaurant,Polish Restaurant,Outdoors & Recreation,Cosmetics Shop


In [60]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 4, 0, 0, 4, 0, 0], dtype=int32)

In [61]:
# add clustering labels
ny_neighborhoods_venues_sorted.insert(0, 'Clustering Label', kmeans.labels_)

ny_merged = ny_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(ny_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Pharmacy,Laundromat,Donut Shop,Pizza Place,Sandwich Place,Caribbean Restaurant,Ice Cream Shop,Gas Station,Dessert Shop,Falafel Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,4,Baseball Field,Bus Station,Restaurant,Park,Chinese Restaurant,Grocery Store,Bagel Shop,Pharmacy,Gift Shop,Mattress Store
2,Bronx,Eastchester,40.887556,-73.827806,4,Caribbean Restaurant,Diner,Intersection,Fast Food Restaurant,Bakery,Donut Shop,Chinese Restaurant,Bowling Alley,Pizza Place,Platform
3,Bronx,Fieldston,40.895437,-73.905643,4,Bus Station,River,Plaza,Women's Store,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Baseball Field,Plaza,Medical Supply Store,Bank,Gym,Playground,Home Service,Food Truck,Bus Station


In [62]:
ny_merged=ny_merged.drop([207,257])

In [63]:
ny_merged['Clustering Label']

0      4
1      4
2      4
3      4
4      0
5      4
6      0
7      4
8      4
9      4
10     0
11     4
12     0
13     4
14     4
15     4
16     0
17     4
18     4
19     4
20     4
21     4
22     0
23     4
24     0
25     4
26     4
27     0
28     4
29     0
30     4
31     4
32     4
33     4
34     4
35     0
36     4
37     0
38     4
39     4
40     4
41     4
42     4
43     4
44     4
45     4
46     0
47     0
48     4
49     0
50     4
51     0
52     0
53     4
54     4
55     4
56     0
57     0
58     0
59     0
60     0
61     0
62     0
63     0
64     0
65     0
66     0
67     0
68     0
69     0
70     0
71     4
72     4
73     4
74     4
75     4
76     0
77     0
78     4
79     0
80     4
81     0
82     0
83     4
84     0
85     0
86     0
87     0
88     4
89     4
90     4
91     0
92     4
93     4
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
1

In [64]:
ny_merged['Clustering Label']=ny_merged['Clustering Label'].astype(int)

In [65]:
ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Pharmacy,Laundromat,Donut Shop,Pizza Place,Sandwich Place,Caribbean Restaurant,Ice Cream Shop,Gas Station,Dessert Shop,Falafel Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,4,Baseball Field,Bus Station,Restaurant,Park,Chinese Restaurant,Grocery Store,Bagel Shop,Pharmacy,Gift Shop,Mattress Store
2,Bronx,Eastchester,40.887556,-73.827806,4,Caribbean Restaurant,Diner,Intersection,Fast Food Restaurant,Bakery,Donut Shop,Chinese Restaurant,Bowling Alley,Pizza Place,Platform
3,Bronx,Fieldston,40.895437,-73.905643,4,Bus Station,River,Plaza,Women's Store,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Baseball Field,Plaza,Medical Supply Store,Bank,Gym,Playground,Home Service,Food Truck,Bus Station


Let's see how this looks on a map.

In [66]:
# create map
map_ny_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Clustering Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_ny_clusters)
       
map_ny_clusters

Toronto

In [68]:
Toronto_main=outcome[outcome['Borough'].str.contains("Toronto")]
Toronto_main.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [69]:
Toronto_venues = getNearbyVenues(names=Toronto_main['Neighbourhood'],
                                   latitudes=Toronto_main['Latitude'],
                                   longitudes=Toronto_main['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [70]:
print(Toronto_venues.shape)
Toronto_venues.head() 

(1200, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [71]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17


Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [72]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


In [73]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot= Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [75]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.04,0.0,0.0,0.00,0.0,0.04,0.0000,0.00,0.0,0.00,0.000000,0.0,0.040000,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0000,0.0,0.0000,0.02,0.020000,0.0000,0.0,0.0,0.020000,0.0,0.0,0.060000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.040000,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.0000,0.02,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.02,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.0200,0.0,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0,0.04,0.0,0.0,0.02,0.0,0.02,0.00,0.00,0.000000,0.000000,0.0200,0.0000,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.000000,0.000000,0.02,0.02,0.0,0.02,0.0,0.0,0.0,0.0000,0.00,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.02,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.0400,0.0000,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.0000,0.0000,0.040000,0.0,0.0,0.0,0.02,0.02,0.0,0.0000,0.04,0.0,0.0,0.0000,0.0,0.02,

In [76]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0         Café  0.06
1   Restaurant  0.04
2  Pizza Place  0.04
3        Hotel  0.04
4    Gastropub  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.04
2         Cheese Shop  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4   Grocery Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0               Spa  0.06
1     Auto Workshop  0.06
2       Pizza Place  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  freq
0       Airport Lounge  0.12
1     Airport Terminal  0.12
2                Plane  0.06
3  Rental Car

In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Seafood Restaurant,American Restaurant,Bar,Steakhouse,Restaurant,Asian Restaurant,Gastropub,Hotel,Pizza Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Garden,Comic Shop,Fast Food Restaurant,Recording Studio,Restaurant,Auto Workshop,Burrito Place,Gym / Fitness Center,Spa,Pizza Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar,Boutique


In [79]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Clustering Label', kmeans.labels_)

Toronto_merged = Toronto_main

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Pizza Place,Brewery,Bubble Tea Shop
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pet Store,Intersection,Steakhouse,Restaurant,Sushi Restaurant,Pub,Brewery,Liquor Store,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Yoga Studio,Diner,Ice Cream Shop,Comfort Food Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Photography Studio,Park,Swim School,Bus Line,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [82]:
# create map
map_tro_clusters = folium.Map(location=[43.40,-79.25], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Clustering Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_tro_clusters)
       
map_tro_clusters

Los Angeles

In [84]:
Los_venues = getNearbyVenues(names=la_data['Neighborhood'],
                                   latitudes=la_data['Latitude'],
                                   longitudes=la_data['Longitude']
                                  )

Agoura Hills
Alhambra
Arcadia
Artesia
Avalon
Azusa
Baldwin Park
Bell
Bell Gardens
Bellflower
Beverly Hills
Bradbury
Burbank
Calabasas
Carson
Cerritos
Claremont
Commerce
Compton
Covina
Cudahy
Culver City
Diamond Bar
Downey
Duarte
El Monte
El Segundo
Gardena
Glendale
Glendora
Hawaiian Gardens
Hawthorne
Hermosa Beach
Hidden Hills
Huntington Park
Inglewood
Irwindale
La Ca?ada Flintridge
La Habra Heights
La Mirada
La Puente
La Verne
Lakewood
Lancaster
Lawndale
Lomita
Long Beach
Los Angeles
Lynwood
Malibu
Manhattan Beach
Maywood
Monrovia
Montebello
Monterey Park
Norwalk
Palmdale
Palos Verdes Estates
Paramount
Pasadena
Pico Rivera
Pomona
Rancho Palos Verdes
Redondo Beach
Rolling Hills
Rolling Hills Estates
Rosemead
San Dimas
San Fernando
San Gabriel
San Marino
Santa Clarita
Santa Fe Springs
Santa Monica
Sierra Madre
Signal Hill
South El Monte
South Gate
South Pasadena
Temple City
Torrance
Walnut
West Covina
West Hollywood
Westlake Village
Whittier


In [85]:
print(Los_venues.shape)
Los_venues.head()

(2152, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agoura Hills,34.136,-118.775,Charm Thai Cuisine,34.136797,-118.774384,Thai Restaurant
1,Alhambra,34.095,-118.127,Blaze Pizza,34.095420,-118.125894,Pizza Place
2,Alhambra,34.095,-118.127,Sprouts Farmers Market,34.095288,-118.125046,Grocery Store
3,Alhambra,34.095,-118.127,Grill 'Em All,34.095595,-118.126725,Burger Joint
4,Alhambra,34.095,-118.127,Borneo Kalimantan Cuisine,34.094818,-118.126659,Food


In [86]:
Los_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agoura Hills,1,1,1,1,1,1
Alhambra,50,50,50,50,50,50
Arcadia,2,2,2,2,2,2
Artesia,38,38,38,38,38,38
Avalon,47,47,47,47,47,47
Azusa,24,24,24,24,24,24
Baldwin Park,36,36,36,36,36,36
Bell,18,18,18,18,18,18
Bell Gardens,5,5,5,5,5,5


In [87]:
print('There are {} uniques categories.'.format(len(Los_venues['Venue Category'].unique())))

There are 256 uniques categories.


In [88]:
# one hot encoding
Los_onehot = pd.get_dummies(Los_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Los_onehot['Neighborhood'] =Los_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Los_onehot.columns[-1]] + list(Los_onehot.columns[:-1])
Los_onehot= Los_onehot[fixed_columns]

Los_onehot.head()

,Zoo,ATM,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Spot,Doctor's Office,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yoshoku Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agoura Hills,0,0,0,0,0,0,0,

In [90]:
Los_onehot.shape

(2152, 256)

In [92]:
Los_grouped = Los_onehot.groupby('Neighborhood').mean().reset_index()
Los_grouped.head()

,Neighborhood,Zoo,ATM,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Spot,Doctor's Office,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yoshoku Restaurant
0,Agoura Hills,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [93]:
num_top_venues = 5

for hood in Los_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Los_grouped[Los_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agoura Hills----
             venue  freq
0  Thai Restaurant   1.0
1              Zoo   0.0
2    Movie Theater   0.0
3           Museum   0.0
4      Music Store   0.0


----Alhambra----
          venue  freq
0        Bakery  0.06
1          Café  0.04
2  Cocktail Bar  0.04
3  Burger Joint  0.04
4        Market  0.04


----Arcadia----
                           venue  freq
0                           Food   0.5
1                       Wine Bar   0.5
2                            Zoo   0.0
3  Paper / Office Supplies Store   0.0
4                         Museum   0.0


----Artesia----
                   venue  freq
0      Indian Restaurant  0.29
1     Chinese Restaurant  0.11
2          Grocery Store  0.08
3        Bubble Tea Shop  0.05
4  Vietnamese Restaurant  0.05


----Avalon----
                venue  freq
0               Hotel  0.19
1  Seafood Restaurant  0.09
2      Ice Cream Shop  0.04
3  Mexican Restaurant  0.04
4                 Bar  0.04


----Azusa----
                venue



----La Verne----
                  venue  freq
0   American Restaurant  0.10
1                 Diner  0.05
2                  Food  0.05
3  Gym / Fitness Center  0.05
4        Farmers Market  0.05


----Lakewood----
                  venue  freq
0            Steakhouse  0.07
1  Fast Food Restaurant  0.07
2        Sandwich Place  0.07
3            Restaurant  0.03
4           Coffee Shop  0.03


----Lancaster----
                      venue  freq
0        Mexican Restaurant  0.14
1              Burger Joint  0.05
2                    Museum  0.05
3  Mediterranean Restaurant  0.05
4           Thai Restaurant  0.05


----Lawndale----
                 venue  freq
0  Peruvian Restaurant  0.07
1         Noodle House  0.07
2         Intersection  0.07
3         Liquor Store  0.07
4       Ice Cream Shop  0.04


----Lomita----
                venue  freq
0         Pizza Place  0.12
1    Asian Restaurant  0.06
2         Video Store  0.06
3  Chinese Restaurant  0.06
4     Thai Restaurant  0.06


In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Los_grouped['Neighborhood']

for ind in np.arange(Los_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Los_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,Thai Restaurant,Yoshoku Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1,Alhambra,Bakery,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,Café,Market,Ice Cream Shop,Burger Joint,Bookstore,Pizza Place
2,Arcadia,Food,Wine Bar,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
3,Artesia,Indian Restaurant,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Vietnamese Restaurant,Market,Thai Restaurant,Tea Room,Taiwanese Restaurant,Frozen Yogurt Shop
4,Avalon,Hotel,Seafood Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Burger Joint,Bar,American Restaurant,Caribbean Restaurant,Candy Store


In [105]:
# set number of clusters
kclusters = 5

Los_grouped_clustering = Los_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Los_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 3, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [106]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Clustering Label', kmeans.labels_)

Los_merged = la_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Los_merged = Los_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Los_merged # check the last columns!

,Neighborhood,Latitude,Longitude,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,34.13600,-118.77500,4.0,Thai Restaurant,Yoshoku Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1,Alhambra,34.09500,-118.12700,1.0,Bakery,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,Café,Market,Ice Cream Shop,Burger Joint,Bookstore,Pizza Place
2,Arcadia,33.98095,-118.14697,3.0,Food,Wine Bar,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
3,Artesia,33.86607,-118.08294,1.0,Indian Restaurant,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Vietnamese Restaurant,Market,Thai Restaurant,Tea Room,Taiwanese Restaurant,Frozen Yogurt Shop
4,Avalon,33.34300,-118.32800,1.0,Hotel,Seafood Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Burger Joint,Bar,American Restaurant,Caribbean Restaurant,Candy Store
5,Azusa,34.13400,-117.90800,1.0,Mexican Restaurant,Pizza Place,Pharmacy,Big Box Store,Bank,Coffee Shop,Japanese Restaurant,Burger Joint,Business Service,Spa
6,Baldwin Park,34.08501,-117.96106,1.0,Mexican Restaurant,Ice Cream Shop,Discount Store,Pharmacy,Fast Food Restaurant,Pizza Place,Bank,Bubble Tea Shop,Japanese Restaurant,Taco Place
7,Bell,33.97800,-118.18700,1.0,Mexican Restaurant,Discount Store,South American Restaurant,Latin American Restaurant,Shoe Store,Burger Joint,Bank,Restaurant,Automotive Shop,Grocery Store
8,Bell Gardens,33.96507,-118.15092,1.0,Fast Food Restaurant,Seafood Restaurant,Mexican Restaurant,Fried Chicken Joint,Latin American Restaurant,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
9,Bellflower,33.88200,-118.11700,0.0,Trail,Concert Hall,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant


In [107]:
Los_merged=Los_merged.drop([64,84])

In [108]:
Los_merged['Clustering Label']

0     4.0
1     1.0
2     3.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     0.0
10    1.0
11    0.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    1.0
33    2.0
34    1.0
35    1.0
36    1.0
37    1.0
38    0.0
39    1.0
40    1.0
41    1.0
42    1.0
43    1.0
44    1.0
45    1.0
46    1.0
47    1.0
48    1.0
49    0.0
50    1.0
51    1.0
52    1.0
53    1.0
54    1.0
55    1.0
56    1.0
57    1.0
58    1.0
59    1.0
60    1.0
61    1.0
62    3.0
63    3.0
65    0.0
66    1.0
67    1.0
68    1.0
69    1.0
70    1.0
71    0.0
72    1.0
73    1.0
74    1.0
75    1.0
76    1.0
77    0.0
78    1.0
79    1.0
80    1.0
81    1.0
82    1.0
83    1.0
85    1.0
Name: Clustering Label, dtype: float64

In [109]:
Los_merged['Clustering Label']=Los_merged['Clustering Label'].astype(int)

In [110]:
Los_merged.head()

,Neighborhood,Latitude,Longitude,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,34.13600,-118.77500,4,Thai Restaurant,Yoshoku Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1,Alhambra,34.09500,-118.12700,1,Bakery,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,Café,Market,Ice Cream Shop,Burger Joint,Bookstore,Pizza Place
2,Arcadia,33.98095,-118.14697,3,Food,Wine Bar,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
3,Artesia,33.86607,-118.08294,1,Indian Restaurant,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Vietnamese Restaurant,Market,Thai Restaurant,Tea Room,Taiwanese Restaurant,Frozen Yogurt Shop
4,Avalon,33.34300,-118.32800,1,Hotel,Seafood Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Burger Joint,Bar,American Restaurant,Caribbean Restaurant,Candy Store


In [111]:
# create map
map_los_clusters = folium.Map(location=[34.03, -118.15], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Los_merged['Latitude'], Los_merged['Longitude'],Los_merged['Neighborhood'], Los_merged['Clustering Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_los_clusters)
       
map_los_clusters

### Cluster examination

NY city

Cluster 1

In [113]:
ny_merged.loc[ny_merged['Clustering Label'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Park,Baseball Field,Plaza,Medical Supply Store,Bank,Gym,Playground,Home Service,Food Truck,Bus Station
6,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Diner,Shopping Mall,Seafood Restaurant,Miscellaneous Shop,Donut Shop,Tennis Stadium
10,Baychester,Donut Shop,Supermarket,Gym / Fitness Center,Music Venue,Men's Store,Sandwich Place,Bank,Mexican Restaurant,Other Great Outdoors,Laundromat
12,City Island,Harbor / Marina,Thrift / Vintage Store,Boat or Ferry,Seafood Restaurant,Pharmacy,French Restaurant,Smoke Shop,Grocery Store,Music Venue,Park
16,Fordham,Mobile Phone Shop,Bank,Shoe Store,Supplement Shop,Donut Shop,Clothing Store,Latin American Restaurant,Fried Chicken Joint,Video Game Store,Diner
22,Port Morris,Latin American Restaurant,Furniture / Home Store,Storage Facility,Restaurant,Distillery,Donut Shop,Peruvian Restaurant,Paper / Office Supplies Store,Spanish Restaurant,Food
24,Hunts Point,Juice Bar,Restaurant,Home Service,Gourmet Shop,Bank,Spanish Restaurant,Waste Facility,Grocery Store,Farmers Market,Café
27,Clason Point,Park,Pool,Grocery Store,Bus Stop,Boat or Ferry,Spa,South American Restaurant,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant
29,Country Club,Sandwich Place,Playground,Women's Store,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory
35,Spuyten Duyvil,Park,Bank,Food,Thai Restaurant,Tennis Stadium,Pharmacy,Asian Restaurant,Farm,English Restaurant,Entertainment Service


Cluster 2

In [114]:
ny_merged.loc[ny_merged['Clustering Label'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Women's Store,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
203,Todt Hill,Park,Women's Store,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


Cluster 3

In [115]:
ny_merged.loc[ny_merged['Clustering Label'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
179,Neponsit,Beach,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


Cluster 4

In [119]:
ny_merged.loc[ny_merged['Clustering Label'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 5

In [118]:
ny_merged.loc[ny_merged['Clustering Label'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Laundromat,Donut Shop,Pizza Place,Sandwich Place,Caribbean Restaurant,Ice Cream Shop,Gas Station,Dessert Shop,Falafel Restaurant
1,Co-op City,Baseball Field,Bus Station,Restaurant,Park,Chinese Restaurant,Grocery Store,Bagel Shop,Pharmacy,Gift Shop,Mattress Store
2,Eastchester,Caribbean Restaurant,Diner,Intersection,Fast Food Restaurant,Bakery,Donut Shop,Chinese Restaurant,Bowling Alley,Pizza Place,Platform
3,Fieldston,Bus Station,River,Plaza,Women's Store,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory
5,Kingsbridge,Pizza Place,Sandwich Place,Mexican Restaurant,Supermarket,Spanish Restaurant,Fried Chicken Joint,Donut Shop,Bakery,Latin American Restaurant,Bar
7,Woodlawn,Pub,Pizza Place,Deli / Bodega,Playground,Convenience Store,Donut Shop,Bar,Trail,Liquor Store,Train Station
8,Norwood,Pizza Place,Park,Bank,Pharmacy,Deli / Bodega,American Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Mexican Restaurant
9,Williamsbridge,Caribbean Restaurant,Bar,Soup Place,Nightclub,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
11,Pelham Parkway,Pizza Place,Italian Restaurant,Frozen Yogurt Shop,Coffee Shop,Bank,Donut Shop,Gas Station,Performing Arts Venue,Sushi Restaurant,Metro Station
13,Bedford Park,Deli / Bodega,Diner,Mexican Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Pharmacy,Discount Store,Supermarket,Bar


Toronto

Cluster 1

In [120]:
Toronto_merged.loc[Toronto_merged['Clustering Label'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Trail,Pub,Health Food Store,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Pizza Place,Brewery,Bubble Tea Shop
42,East Toronto,0,Sandwich Place,Pet Store,Intersection,Steakhouse,Restaurant,Sushi Restaurant,Pub,Brewery,Liquor Store,Fast Food Restaurant
43,East Toronto,0,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Yoga Studio,Diner,Ice Cream Shop,Comfort Food Restaurant
45,Central Toronto,0,Food & Drink Shop,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Department Store,Doner Restaurant,Dog Run,Distribution Center
46,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Restaurant,Mexican Restaurant,Café,Salon / Barbershop,Chinese Restaurant,Yoga Studio,Dessert Shop
47,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Café,Italian Restaurant,Sushi Restaurant,Coffee Shop,Gas Station,Diner
49,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Bank,Sports Bar,Restaurant,Liquor Store,Pizza Place,American Restaurant,Light Rail Station
51,Downtown Toronto,0,Coffee Shop,Park,Bakery,Restaurant,Italian Restaurant,Pub,Chinese Restaurant,Pizza Place,Café,Gastropub
52,Downtown Toronto,0,Gastropub,Gay Bar,Coffee Shop,Men's Store,Restaurant,Pub,Pizza Place,Park,Nightclub,Mexican Restaurant


Cluster 2

In [121]:
Toronto_merged.loc[Toronto_merged['Clustering Label'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Tennis Court,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Cluster 3

In [122]:
Toronto_merged.loc[Toronto_merged['Clustering Label'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Playground,Trail,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Cluster 4

In [123]:
Toronto_merged.loc[Toronto_merged['Clustering Label'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Music Venue,Wings Joint,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Cluster 5

In [124]:
Toronto_merged.loc[Toronto_merged['Clustering Label'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Clustering Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Photography Studio,Park,Swim School,Bus Line,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Los Angeles

Cluster 1 

In [125]:
Los_merged.loc[Los_merged['Clustering Label'] == 0, Los_merged.columns[[1] + list(range(5, Los_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,33.88200,Concert Hall,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
11,34.14695,Home Service,Fast Food Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Donut Shop,Farmers Market
38,33.96100,Home Service,Yoshoku Restaurant,Dry Cleaner,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
49,34.02600,Beach,Harbor / Marina,Yoshoku Restaurant,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm
65,33.78792,Park,Farm,Donut Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
71,34.39200,Home Service,Construction & Landscaping,Carpet Store,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
77,33.95500,Park,Liquor Store,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Dry Cleaner


Cluster 2 

In [126]:
Los_merged.loc[Los_merged['Clustering Label'] == 1, Los_merged.columns[[1] + list(range(5, Los_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,34.09500,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,Café,Market,Ice Cream Shop,Burger Joint,Bookstore,Pizza Place
3,33.86607,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Vietnamese Restaurant,Market,Thai Restaurant,Tea Room,Taiwanese Restaurant,Frozen Yogurt Shop
4,33.34300,Seafood Restaurant,Mexican Restaurant,Pizza Place,Ice Cream Shop,Burger Joint,Bar,American Restaurant,Caribbean Restaurant,Candy Store
5,34.13400,Pizza Place,Pharmacy,Big Box Store,Bank,Coffee Shop,Japanese Restaurant,Burger Joint,Business Service,Spa
6,34.08501,Ice Cream Shop,Discount Store,Pharmacy,Fast Food Restaurant,Pizza Place,Bank,Bubble Tea Shop,Japanese Restaurant,Taco Place
7,33.97800,Discount Store,South American Restaurant,Latin American Restaurant,Shoe Store,Burger Joint,Bank,Restaurant,Automotive Shop,Grocery Store
8,33.96507,Seafood Restaurant,Mexican Restaurant,Fried Chicken Joint,Latin American Restaurant,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
10,34.07400,Boutique,Park,Salon / Barbershop,American Restaurant,Hotel,Coffee Shop,Café,Health & Beauty Service,Sandwich Place
12,34.18100,Japanese Restaurant,Pizza Place,Sandwich Place,Ice Cream Shop,Clothing Store,Mexican Restaurant,Sports Bar,Spa,Brazilian Restaurant
13,34.15800,Juice Bar,Ice Cream Shop,Fast Food Restaurant,Sushi Restaurant,Seafood Restaurant,Intersection,Shipping Store,ATM,Cosmetics Shop


Cluster 3 

In [127]:
Los_merged.loc[Los_merged['Clustering Label'] == 2, Los_merged.columns[[1] + list(range(5, Los_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,34.16409,Hotel,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm


Cluster 4

In [128]:
Los_merged.loc[Los_merged['Clustering Label'] == 3, Los_merged.columns[[1] + list(range(5, Los_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,33.98095,Wine Bar,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
62,33.74400,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
63,33.74400,Dry Cleaner,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm


Cluster 5 

In [129]:
Los_merged.loc[Los_merged['Clustering Label'] == 4, Los_merged.columns[[1] + list(range(5, Los_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,34.136,Yoshoku Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market


## Results and Discussion <a name="results"></a>

The major clustering in the New York city,as we examine the clustering,we can find that it is the most prosperous cluster in the New York city.

## Conclusion <a name="conclusion"></a>

We can see from our analysis that New York city and Los Angeles are much more similiar and it may be due to the country and latitude.